In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import glob
import cPickle as pickle
import corner

from astropy.io import fits, ascii
from astropy.time import Time
from collections import Counter
from matplotlib.colors import LogNorm
from matplotlib.widgets import Slider
from scipy.stats import mode

from __future__ import division

In [2]:
%matplotlib qt5
plt.ion()

In [3]:
sns.set_context('paper')
sns.set_style('darkgrid')
sns.set_color_codes('deep')

## Wavelength Calibration

Some of the lamp spectra through the target and comparison star slits are completely shifted off of the reference lamp spectra, so guess_lines.py can't be used. Instead, the lines will be indentified manually from http://www.lco.cl/telescopes-information/magellan/instruments/imacs/imacs-spectral-atlas/imacs-spectral-atlas.

To make things a little easier, this routine will will automatically record the pixel coordinate from a mouse press to save me an extra step in writing down the pixel/wavelength values.

In [45]:
# read in desired lamp spec
name = 'WASP43b_8'
path = '/Users/mango/Desktop/arcs_ut150224/' + name + '_arc.fits'
#path = '/Users/mango/Desktop/HeNeAr_gris300.fits'
hdu = fits.open(path)
data = hdu[0].data
hdu.close()

pix_loc = []

fig, ax = plt.subplots(figsize = (15,4))

#ax.cla()
ax.plot(data)
ax.set_title(name)
ax.set_xlabel('pixel')
ax.set_ylabel('flux (arbitrary units)')
#[ax.axvline(xc, lw=0.5, ls = '--', c='r', alpha=0.5) for xc in arc_lines['pix']]

def on_key(event): # key press
    if (event.key == 'x' or event.key == 'X') and event.inaxes == ax:
        x = event.xdata
        y = event.ydata
        
        plt.axvline(x, c='r', ls='--', lw=0.5)
        ax.plot(x, y, 'rX', ms=10)
        x = str( np.round(x, 4) )
        pix_loc.append(x)
        
    fig.canvas.draw()

    
fig.tight_layout()
cid = fig.canvas.mpl_connect('key_press_event', on_key)
#wav = str( round(input("Wav: "), 4) )

In [46]:
f = open('/Users/mango/Desktop/test.txt', 'wb')
for pix in pix_loc:
    f.write("%s\n" % pix)
f.close()

### noao line lines

In [41]:
plt.figure(figsize=(15,4))
arc_lines = ascii.read('/Users/mango/Dropbox/mercedes-group/wavelength_calibration/noao/ref_linelist.txt')
hdu = fits.open('/Users/mango/Dropbox/mercedes-group/wavelength_calibration/noao/noao.fits')

data = hdu[0].data
hdu.close()

wavs = arc_lines['Fit']
xs = arc_lines['Pixel']
ys = []
for x_guess in xs:
    ys.append(data[int(x_guess)])
plt.plot(data, c='orange')
[plt.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
[plt.annotate(str(wav), (s_x, s_y)) for wav, s_x, s_y in zip(wavs, xs, ys)]

plt.title('noao reference')
plt.xlabel('pixel')
plt.ylabel('arbitrary flux')

plt.tight_layout()

plt.show()

### lco lamp lines

In [44]:
plt.figure(figsize=(15,4))
arc_lines = ascii.read('/Users/mango/Dropbox/mercedes-group/wavelength_calibration/lco_lines.txt')

hdu = fits.open('/Users/mango/Dropbox/mercedes-group/wavelength_calibration/HeNeAr_gris300.fits')
data = hdu[0].data
hdu.close()

wavs = arc_lines['wav']
xs = arc_lines['pix']
ys = []
for x_guess in xs:
    ys.append(data[int(x_guess)])
plt.plot(data, c='orange')
[plt.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
[plt.annotate(str(wav), (s_x, s_y)) for wav, s_x, s_y in zip(wavs, xs, ys)]

plt.title('lco reference')
plt.xlabel('pixel')
plt.ylabel('arbitrary flux')

plt.tight_layout()

plt.show()

### calibration file check

In [70]:
plt.figure(figsize=(15,4))

guess_lines = ascii.read('/Users/mango/Desktop/arcs_ut150224/comp6_lines_chips.txt')

hdu = fits.open('/Users/mango/Desktop/arcs_ut150224/stage/comp6_5_arc.fits')
data = hdu[0].data
hdu.close()

wavs = guess_lines['Wav']
xs = guess_lines['Pix']
ys = []
for x_guess in xs:
    ys.append(data[int(x_guess)])

plt.plot(data, c='g')
[plt.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
[plt.annotate(str(wav), (s_x, s_y)) for wav, s_x, s_y in zip(wavs, xs, ys)]

plt.xlabel('pixel')
plt.ylabel('arbitrary flux')

plt.tight_layout()

plt.show()

### Overview Line List Inspection

In [50]:
s = '/Users/mango/Dropbox/mercedes-group/wavelength_calibration/arcs_ut150224/comp1_2_arc.fits'

In [54]:
s.split('/')[-1]

'comp1_2_arc.fits'

In [71]:
fig, axis = plt.subplots(2, 4, figsize = (14,5))

ax_list = []
for i in range(2):
    for j in range(4):
        ax_list.append(axis[i][j])

#read in spectra and line lists
spec_list = []
line_lists = []
title_list = []
for i in glob.glob('/Users/mango/Desktop/arcs_ut150224/stage/*arc.fits'):
    hdu = fits.open(i)
    spec = hdu[0].data # data
    spec_list.append(spec)
    title_list.append(i.split('/')[-1])
    hdu.close()
    
for i in glob.glob('/Users/mango/Desktop/arcs_ut150224/*lines*txt'):
    lines = ascii.read(i)
    line_lists.append(lines)

k = 0
z = 0
xs_list = []
for ax, spec, lines, ti in zip(ax_list, spec_list, line_lists, title_list):
    wavs = lines['Wav']
    xs = lines['Pix']
    xs_list.append(xs)
    ys = []
    for x_guess in xs:
        ys.append(spec[int(x_guess)])

    ax.plot(spec, c='orange')
    ax.set_title(ti)
    [ax.axvline(xc, lw=0.5, ls = '--', c='b', alpha=0.5) for xc in xs]
    [ax.annotate(str(wav), (s_x, s_y), fontsize=12) for wav, s_x, s_y in zip(wavs, xs, ys)]

    i = 0

def on_key(event): # pan and zoom each plot key press
    global i
    if (event.key == 'n' or event.key == 'N'):
        for ax, spec, lines in zip(ax_list, spec_list, line_lists):
            wavs = lines['Wav']
            xs = lines['Pix']
            xs_list.append(xs)
            ys = []
            for x_guess in xs:
                ys.append(spec[int(x_guess)])

            ax.set_xlim(xs[i] - 50, xs[i] + 50)
            ax.set_ylim(ys[i]*-0.1, ys[i]*3)

        i += 1
        if (i > len(xs) - 1): # go back to beginning after reaching end
            i = 0
            

    fig.canvas.draw()


fig.tight_layout()
cid = fig.canvas.mpl_connect('key_press_event', on_key)

## File Inspection

### trace check

In [ ]:
import os

In [ ]:
os.chdir('/Users/mango/tepspec/IMACS/')

In [ ]:
obj_xx = pickle.load(open("/Users/mango/Desktop/XX.pkl","rb"))
obj_yy = pickle.load(open("/Users/mango/Desktop/YY.pkl","rb"))

In [ ]:
import sys
sys.path.append("../ExtraCodes/BaryCor/")

In [ ]:
#import TEPSPEC_Util
obj_trace = pickle.load(open("./Objects_trace.pkl","rb"))

In [ ]:
obj_trace['WASP43b_8'].traces[0]

In [ ]:
obj = 'WASP43b_8'

[plt.plot(obj_xx[obj][i], obj_yy[obj][i], '.', label=i) for i in range(18, 26)]

plt.legend()
plt.xlabel('X')
plt.ylabel('Y')

#plt.savefig("/Users/mango/Desktop/raw_traces.png", bbox_inches='tight')

### spec files

In [ ]:
"""
plots items in <object>_spec.fits files.
spec_item:
0: Wavelength
1: Simple extracted object spectrum
2: Simple extracted flat spectrum
3: Pixel sensitivity (obtained by the flat)
4: Simple extracted object spectrum/pixel sensitivity
5: Sky flag (0 = note uneven sky, 1 = probably uneven profile, 2 = Sky_Base failed)
6: Optimally extracted pbject spectrum
7: Optimally extracted object spectrum/pixel sensitivity
"""
def spec_plot(spec_item):
    fig, axs = plt.subplots(1,2,figsize=(10,3))
    sns.set_style("darkgrid")

    objs = ['comp4_4', 'comp4_4']
    for ax, obj in zip(axs, objs):
        hdu = fits.open('/Users/mango/Desktop/'+obj+'_spec.fits')
        data = hdu[0].data
        hdu.close()
        
        # data[number of sci frames, spec_items, pixel height of spectrum (2048)]
        # by 'height' I mean in spectral direction
        for i in range(len(data[:,0,0])):
            ax.plot(data[i,spec_item,:])
        
        #ax.set_title(obj.split('_')[-1])
        
    fig.tight_layout()
    fig.show()

spec_plot(0)

In [ ]:
path_list = glob.glob(
    '/Users/mango/data/ACCESS/IMACS/WASP43/ut150309/*0*.fits')

object_dict = {}
am = []
for path in path_list:
    hdu = fits.open(path)
    filename = hdu[0].header['FILENAME']
    obj     = hdu[0].header['OBJECT']
    airmass = hdu[0].header['AIRMASS']
    am.append(airmass)
    hdu.close()
    if ('c8' in filename and airmass < 1.5 and 'science' in obj): # other chips redundant for ID purposes
        object_dict.update({filename:obj})

#for key in sorted(object_dict.iterkeys()):
#    print key, object_dict[key]

In [ ]:
import re
sc_counter = 0
sci_frame = []
for k, v in object_dict.iteritems():
    if 'science' in v:
        sc_counter += 1
        file_num = re.split('t|c', k) # get number portion of filename
        sci_frame.append(file_num[1])
sci_frame = np.sort(sci_frame)
print sc_counter

In [ ]:
object_dict

### For adding full flats from WASP-19 to WASP-43 ut170410 dataset

In [ ]:
full_flats = []
for i, j in object_dict.items():
    if 'full' in j and 'c8' in i:
        print i,j
        full_flats.append(i)

#### Airmass filter

In [ ]:
path_list = glob.glob(
    '/data/ACCESS/IMACS/WASP43/ut150224/*0*.fits')

filename_list = []
for path in path_list:
    hdu = fits.open(path)
    filename = hdu[0].header['FILENAME']
    airmass = hdu[0].header['AIRMASS']
    obj     = hdu[0].header['OBJECT']
    hdu.close()
    #if obj == 'science image':
    #    i += 1
    #print obj, filename
    if ((airmass >= 1.5) and ('c8' in filename) and 'sci' in obj): #and (obj == 'science image'):
        #filename_list.append(filename)
        print filename, obj
    #if ('science frame wasp43' in obj and 'c8' in filename):
    #    filename_list.append(filename)

## Trace Centroid Check

In [ ]:
xx = pickle.load(open('/Users/mango/Desktop/XX.pkl', 'r'))
yy = pickle.load(open('/Users/mango/Desktop/YY.pkl', 'r'))

In [ ]:
target_dict={'w8':'WASP43b_8', 'w3':'WASP43b_3',
             'c12':'comp1_2',  'c15':'comp1_5',
             'c28':'comp2_8',  'c23':'comp2_3',
             'c36':'comp3_6',  'c31':'comp3_1',
             'c47':'comp4_7',  'c44':'comp4_4',
             'c51':'comp5_1',  'c56':'comp5_6',
             'c65':'comp6_5',  'c62':'comp62'}
target = target_dict['w8']

fig, ax = plt.subplots()

line_list = []
for i in range(len(xx[target])):
    line, = ax.plot(xx[target][i], yy[target][i], alpha=1)
    line_list.append(line)
    
ax_yval = plt.axes([0.3, 0.01, 0.6, 0.03]) # [left, bottom, width, height]
s_yval = Slider(ax_yval, 'y', 0, len(xx[target]), valinit=0)

def update(val): # get value from slider and plugs it into plot
    c = s_yval.val
    for i, l in enumerate(line_list):
        if  i <= c:
            l.set_visible(True)
        else:
            l.set_visible(False)

s_yval.on_changed(update)

plt.show()

## Aperture Check

#### Interactive 1D profile plotter to verify apertures used in tepspec

In [ ]:
path = '/Users/mango/data/ACCESS/IMACS/WASP43/ut150309/ift0308c8.fits'
hdu = fits.open(path)
name = hdu[0].header['filename']
data_25 = hdu[0].data
ut_start = hdu[0].header['UT-TIME']
ut_end = hdu[0].header['UT-END']
hdu.close()

path = '/Users/mango/data/ACCESS/IMACS/WASP43/ut150309/ift0310c8.fits'
hdu = fits.open(path)
name = hdu[0].header['filename']
data_27 = hdu[0].data
ut_start = hdu[0].header['UT-TIME']
ut_end = hdu[0].header['UT-END']
hdu.close()

plt.close()
fig, axes = plt.subplots(1,1, figsize=(6, 8), sharey=True)

im = axes.imshow((data_27 - data_25), cmap='CMRmap', vmin=0, vmax=60)
#norm=LogNorm(vmin = 1E1, vmax=1E2))
plt.colorbar(im, ax=axes)
plt.title('science frame 27 - 25')
#axes.grid(False) # get rid of gridlines on imshow
#axes.set_title(name + ' -- sci frame 25')
#axes.set_xlabel('%s - %s' % (ut_start, ut_end))
#plt.colorbar(im, ax = axes[0])

plt.savefig('/Users/mango/Desktop/ut150309_image.png', bbox_inches='tight')

In [ ]:
plt.plot(data_27[1000][:]/np.max(data_27[1000][:]) - data_29[1000][:]/np.max(data_29[1000][:]))
plt.xlim(670, 740)
plt.title('sci frame 27 - 29')
plt.xlabel('x')
plt.ylabel('relative flux')
#plt.savefig('/Users/mango/Desktop/ut150309_profile.png', bbox_inches='tight')

In [ ]:
plt.plot(data_29[1000][:])

In [ ]:
plt.plot((data_27 - data_29)[1000][:])

In [ ]:
len(xx['WASP43b_8'][27]) #- xx['WASP43b_8'][25]

In [ ]:
len(xx['WASP43b_8'][25])

In [ ]:
def drag(event): # mouse drag
    x = event.xdata
    y = event.ydata
    
    if (event.inaxes == axes[0]): # only detect mouse drags in imshow plot
        axes[1].cla()
        axes[1].plot(data[int(round(y)), :])
        axes[1].set_xlim(x - 25, x + 25)
        axes[1].set_ylabel('counts')
    
    fig.tight_layout()
    fig.canvas.draw()
    
cid = fig.canvas.mpl_connect('motion_notify_event', drag)

# Light Curve

## Load in LC pickles

In [ ]:
thefile = open('WASP43/ut170410/tepspec_final/LCs_w43_170410.pkl', 'rb')
LC_ut170410 = pickle.load(thefile)
thefile.close()

## Raw WLC Summary

In [ ]:
def raw_WLC(ax, LC, dataset):
    t = Time(LC['t'], format='jd')
    
    LC_o = LC['oLC']

    # plot comparison stars next
    for i in range(0,6):
        lightcurve_i = LC['cLC'][:,i]
        lc_i = lightcurve_i/np.median(lightcurve_i)# - (i+1)/2
        #p = ax.plot_date(t.plot_date, lc_i, '-',label='comp %i' % (i+1)) 
        p = ax.plot(lc_i, '-',label='comp %i' % (i+1))
        #ax.annotate( 'comp %i' % (i+1), (t.plot_date[0], lc_i[0]+0.05), 
        #color = p[0].get_color())
    
    # plot target LC first
    LC_o_rel = LC_o/np.median(LC_o)
    #p = ax.plot_date(t.plot_date, LC_o_rel, '-', lw=3, c='k', label = 'WASP43b')
    p = ax.plot(LC_o_rel, '-', lw=3, c='k', label = 'WASP43b')

    #ax.annotate( 'WASP43', (t.plot_date[0], LC_o_rel[0]+0.05), color = 'grey')
    
    ax.set_xlabel('Time (UTC)')
    ax.set_ylabel('Relative Flux - Offset')
    ax.set_title('Raw White Light Curves - ' + dataset)
    ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1)

raw_WLC(ax, LC_ut170410, 'ut170410')
#plt.savefig(
#    '/Users/mango/Dropbox/mercedes-group/journal/Figures/20170709/ut170410_sum.pdf',
#    bbox_inches='tight')

## WLC - Comp Stars

In [ ]:
# comp stars to use, e.g. ['comp1', 'comp2']
def raw_WLC_adj(ax, LC, comp_names): 
    t = Time(LC['t'], format='jd')
    LC_o = LC['oLC']
    
    #########################################################
    # plot 1
    ###########################################
    comp_list = [] # each entry is comparison star flux
    for i in comp_names: 
        # comp1 is index 0 in pickle file, comp2 is 1, etc.
        comp_idx = int(i.split('p')[-1]) - 1
        comp_list.append(LC['cLC'][:,comp_idx])
            
    LC_comps = np.sum(comp_list, axis=0)
    LC_adj = LC_o / LC_comps
    #ax.plot_date(t.plot_date, LC_adj / np.median(LC_adj), '.')
    ax.plot(LC_adj / np.median(LC_adj), '.')

    #plt.axvline(2457090.709 + 0.0308274)
    #plt.ylim(0.99, 1.02)
    ax.set_xlabel('Time (UTC)')
    ax.set_ylabel('Relative Flux')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

raw_WLC_adj(ax, LC_ut170410, ['comp1','comp2','comp3','comp4',
                             'comp6'])
#raw_WLC_adj(ax, LC_ut150309, ['comp1','comp6'])
#raw_WLC_adj(ax, LC_ut150309, ['comp1', 'comp6'])
#ax.set_title('WASP43 -- ut150309 -- tepspec WLC -- ' + 
#    'Spectroscopic2 -- Gri-300-17.5')

#ax.set_ylim(0.97,1.01)
#ax.set_ylim(0.97,1.01)
#plt.savefig(
#    '/Users/mango/Dropbox/mercedes-group/journal/Figures/20170731/ut170410_adjWLC.pdf',
#    bbox_inches='tight')

## LBR Comparison

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8,5), sharex=True, 
                       gridspec_kw = {'height_ratios':[2, 1]})
comp_names = ['comp1','comp2','comp3','comp4','comp5','comp6']

#########################################################
# LBR
###########################################
LC = LC_ut170410_LBR
t = Time(LC['t'], format='jd')
t = t[:-1]
LC_o = LC['oLC']
LC_o = LC_o[:-1]
comp_list = [] # each entry is comparison star flux
for i in comp_names: 
    # comp1 is index 0 in pickle file, comp2 is 1, etc.
    comp_idx = int(i.split('p')[-1]) - 1
    comp_list.append(LC['cLC'][:-1,comp_idx])

LC_comps = np.sum(comp_list, axis=0)
LC_adj = LC_o / LC_comps
ax[0].plot_date(t.plot_date, LC_adj / np.median(LC_adj), 'r.', 
            label='LBR')

#########################################################
# No LBR
###########################################
LC = LC_ut170410
t = Time(LC['t'], format='jd')
t = t[:-1]
LC_o = LC['oLC']
LC_o = LC_o[:-1]
comp_list = [] # each entry is comparison star flux
for i in comp_names: 
    # comp1 is index 0 in pickle file, comp2 is 1, etc.
    comp_idx = int(i.split('p')[-1]) - 1
    comp_list.append(LC['cLC'][:-1,comp_idx])

LC_comps = np.sum(comp_list, axis=0)
LC_adj_2 = LC_o / LC_comps
ax[0].plot_date(t.plot_date, LC_adj_2 / np.median(LC_adj_2), 'b.', 
            label='No LBR')
#ax.plot(LC_adj / np.median(LC_adj), 'b.', label='No LBR')

#plt.axvline(2457090.709 + 0.0308274)
#plt.ylim(0.99, 1.02)
ax[0].set_xlabel('Time (UTC)')
ax[0].set_ylabel('Relative Flux')
ax[0].set_title('tepspec WLC - ut170410')
#ax[0].set_ylim(0.9734, 1.005)
ax[0].legend()

ax[1].plot_date(t.plot_date, LC_adj_2 - LC_adj, 'c.')
ax[1].set_ylabel('No LBR - LBR')

fig.tight_layout()
plt.savefig('/Users/mango/Desktop/LBR_comparison.png', 
            bbox_inches='tight', dpi=1000)

## Spectra Inspection

In [ ]:
def find_nearest_idx(array, value): # get index of array where val is nearest
    idx = (np.abs(array-value)).argmin()
    return idx

def find_min(wav, val, loc): # return location (wav, depth) of spectra near loc
    # first find wavlength of discretized spectra nearest loc
    wav_cen_idx = find_nearest_idx(wav, loc)
    
    # now make a 3.75 Angstrom window (aperture) around center and look for min.
    # this will hopefully be where the true absorption peak is.
    # note: a difference of 1 in index is 1.25 in Angstroms
    peak_idx = np.where(val == val[wav_cen_idx-3:wav_cen_idx+4].min())[0][0]
    # the [0][0] just makes sure we take the first instance of the minimum that is
    # found in the window, in case there are double peaks
    return peak_idx, val[peak_idx]

### Plot single spectra

In [ ]:
def plot_spec(ax, LC, time_idx, c):  
    # naD is a doublet but it looks like a single peak at our resolution so I am
    #just going to take the average for this one
    species = {
    '$NaI-D$'    : {'wav':5892.9, 'c':'b' , 'dif':[], 'ls':'-'} ,
    r'$H\alpha$' : {'wav':6564.6, 'c':'g' , 'dif':[], 'ls':'-'} ,
    '$KI-a$'   : {'wav':7665.0, 'c':'y' , 'dif':[], 'ls':'-'} ,
    '$KI-b$'   : {'wav':7699.0, 'c':'y' , 'dif':[], 'ls':'--'} ,
    '$NaI-8200a$' : {'wav':8183.0, 'c':'r' , 'dif':[], 'ls':'-'} ,
    '$NaI-8200b$' : {'wav':8195.0, 'c':'r' , 'dif':[], 'ls':'--'}
    }
    
    wav    = LC['spectra']['wavelengths']
    transm = LC['spectra']['WASP43b'][time_idx,:]
    t = Time(LC['t'], format='jd')
    
    # plot reference lines and peak points
    for elem, elem_attr in species.items():
        # overplot bins
        #for l, r in zip(wav_list['col1'], wav_list['col2']):
        #    if l in [5891, 6563.5, 7662, 7696, 8179, 8192]:
        #        ax.axvspan(l, r, color='r', alpha=0.3)
        #    else:
        #        ax.axvspan(l, r, color='b', alpha=0.03)
        
        ax.axvline(elem_attr['wav'], ls=elem_attr['ls'], c=elem_attr['c'], label=elem)
        
        pk_idx, pk_val = find_min(wav, transm, elem_attr['wav'])
        ax.plot(wav[pk_idx], pk_val, 'o', color = elem_attr['c'])

    # overplot spectra
    ax.plot(wav, transm, c)
    
    ax.set_title('WASP43 -- ut170410 -- tepspec spectra -- ' + 
    'WB5600-9200 -- Gri-300-17.5', fontsize=12)
    ax.set_xlabel("$\AA$")
    ax.set_ylabel("Depth (arbitrary units)")
    
    handles,labels = ax.get_legend_handles_labels()
    handles = [handles[2], handles[0], handles[5], handles[4], 
               handles[3], handles[1]]
    labels = [labels[2], labels[0], labels[5], labels[4], 
              labels[3], labels[1]]
    ax.legend(handles, labels)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
plot_spec(ax, LC_ut170410, 139, 'c') # "zero"

#plt.savefig('/Users/mango/Desktop/tepspec_spec_t0.pdf', bboc_inches='tight')

### Bin Setup

In [ ]:
# hold observed feature centers, as opposed to vacumm
spec_cen = [5892.5, 6565.0, 7666.25, 7700, 8185, 8196.52] 

for spec in spec_cen:
    l, r = 5, 5
    print "bins for: ", spec
    for i in range(10):
            print spec - l, spec + r
        l += 5
        r += 5
    print ""

In [ ]:
lower = 8246.52
upper = 9198.0
x = np.arange(lower, upper, 250)
for i, j in enumerate(x):
    if (i < len(x)-1):
        print x[i], x[i+1]
    else:
        print x[i], upper

In [ ]:
wav_list = ascii.read('/Users/mango/Desktop/w43_170410.dat')

In [ ]:
fig, ax = plt.subplots(1,1)
for l, r in zip(wav_list['col1'], wav_list['col2']):
    if l in [5891, 6563.5, 7662, 7696, 8179, 8192]:
        ax.axvspan(l, r, color='r', alpha=0.3)
    else:
        ax.axvspan(l, r, color='b', alpha=0.3)
plt.show()

### Plot spec differences

In [ ]:
from matplotlib.dates import  DateFormatter
def plot_spec_diff(LC, comp_names):    
    # naD is a doublet but it looks like a single peak at our resolution so I am
    #just going to take the average for this one
    species = {
    '$NaI-D$'    : {'wav':5892.9, 'c':'b' , 'dif':[], 'ls':'.-'} ,
    r'$H\alpha$' : {'wav':6564.6, 'c':'g' , 'dif':[], 'ls':'.-'} ,
    '$KI-a$'   : {'wav':7665.0, 'c':'y' , 'dif':[], 'ls':'.-'} ,
    '$KI-b$'   : {'wav':7699.0, 'c':'y' , 'dif':[], 'ls':'.--'} ,
    '$NaI-8200a$' : {'wav':8183.0, 'c':'r' , 'dif':[], 'ls':'.-'} ,
    '$NaI-8200b$' : {'wav':8195.0, 'c':'r' , 'dif':[], 'ls':'.--'}
    }
    
    # fill in dictionary 
    for i in range(len(LC['t'])): # iterate through time
        wav    = LC['spectra']['wavelengths']
        transm = LC['spectra']['WASP43b'][i,:]
        t = Time(LC['t'], format='jd')
        
        for elem, elem_attr in species.items(): # iterate through species
            pk_idx, val = find_min(wav, transm, elem_attr['wav'])
            dif = wav[pk_idx] - elem_attr['wav'] # measured - vacuum
            elem_attr['dif'].append(dif)
            
    # plot the differences
    fig, ax = plt.subplots(2, 1, figsize=(8,5), sharex=False)
    
    LC_o = LC['oLC']
    
    #########################################################
    # LC plot
    ###########################################
    comp_dict = {} # hold all comparison star
    for i in range(6):
        comp_dict["comp"+str(i+1)] = LC['cLC'][:,i]

    comp_list = []
    for i in comp_names: # only uses selected comparison stars
        comp_list.append(comp_dict[i])

    LC_comps = np.sum(comp_list, axis=0)
    LC_adj = LC_o / LC_comps
    #ax[0].plot_date(t.plot_date, LC_adj / np.median(LC_adj), 'c.')
    ax[0].plot(LC_adj / np.median(LC_adj), 'c.')
    ax[0].set_ylabel('Arbitrary Flux')
    ax[0].set_title('WASP43 -- ut170410 -- tepspec all comps -- ' + 
    'WB5600-9200 -- Gri-300-17.5', fontsize=12)
    
    #########################################################
    # dif plot
    ###########################################
    for elem, elem_attr in species.items():
        #ax[1].xaxis.set_major_formatter( DateFormatter('%H:%M') )

        ax[1].plot(elem_attr['dif'], elem_attr['c'] + elem_attr['ls'], 
                       label = elem)
        
    handles,labels = ax[1].get_legend_handles_labels()
    handles = [handles[2], handles[0], handles[5], handles[4], 
               handles[3], handles[1]]
    labels = [labels[2], labels[0], labels[5], labels[4], 
              labels[3], labels[1]]
    ax[1].legend(handles, labels, loc=3, ncol=6)
    ax[1].set_xlabel('Time (Index)')
    ax[1].set_ylabel('Measured - Vacuum ($\AA$)')
    ax[1].set_ylim(-2.5,3.5)
    fig.tight_layout()
    plt.show()

In [ ]:
plot_spec_diff(LC_ut170410, 
        ['comp1', 'comp2', 'comp3', 'comp4', 'comp6'])
#plt.savefig('/Users/mango/Desktop/tepspec_diff_spec.pdf',
#            bbox_inches='tight')

# ELVIS

## poly and pca WLC

In [ ]:
def plot_wlc(ax, fn): # plot white light curve
    # load data (date: ut, model: poly of pca, binning: wlc)
    date, model, binning = fn.split(' ')
    path = date + '/' + model + '_' + binning
    thefile = open(
    '/Users/mango/Desktop/'+path+'/final_LCs.pkl', 'rb')
    wlc_data = pickle.load(thefile)
    thefile.close()
    
    # Model values
    wlc_t0   = wlc_data['t0']
    wlc_t0_iso = Time(wlc_t0, format='jd').iso
    print 't0: ', Time(wlc_t0, format='jd').iso
    print 'JD: ', wlc_t0
    wlc_t    = wlc_data['t']
    t_rel = (wlc_t - wlc_t0) * 24. # convert to hours
    wlc_flux = wlc_data['model'][0]

    # detrended values
    wlc_detLC = wlc_data['detLC'][0]

    ax.plot(t_rel, wlc_detLC, 'o', c='lightblue')
    ax.plot(t_rel, wlc_flux, 'b')
    ax.set_ylabel('normalized flux')
    
def plot_resid(ax, fn):
     # load data (date: ut, model: poly of pca, binning: wlc)
    date, model, binning = fn.split(' ')
    path = date + '/' + model + '_' + binning
    thefile = open(
    '/Users/mango/Desktop/'+path+'/final_LCs.pkl', 'rb')
    wlc_data = pickle.load(thefile)
    thefile.close()
    
    # Model values
    wlc_t0   = wlc_data['t0']
    wlc_t    = wlc_data['t']
    t_rel = (wlc_t - wlc_t0) * 24. # convert to hours
    
    ax.plot(t_rel, wlc_data['residuals'][0], 'o', c='pink')
    ax.set_ylabel('residuals')
    ax.set_xlabel('$t - t_0$ (hours)')
    #ax.set_ylim(-0.001, 0.001)

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8,6),
                       gridspec_kw = {'height_ratios':[2, 1]})
title = 'WASP43 -- ut170410 -- PCA polynomial wavelet WLC -- ' + \
    'WB5600-9200 -- Gri-300-17.5'
#title = 'WASP43 -- ut150309 -- PCA polynomial wavelet WLC -- ' + \
#    'Spectroscopic2 -- Gri-300-17.5'
#title = 'WASP43 -- ut150224 -- PCA polynomial wavelet WLC -- ' + \
#'Spectroscopic2 -- Gri-300-17.5'
    
obj = title.split(' ')
obj = "%s %s %s" % (obj[2],obj[4],obj[7])
print obj

ax[0].set_title(title, fontsize=12)
plot_wlc(ax[0], obj)
plot_resid(ax[1], obj)

fig.tight_layout()
#plt.savefig('/Users/mango/Desktop/ut150309_pca_wlc.png',
#           bbox_inches='tight')
#plt.savefig('/Users/mango/Desktop/ut170410_pca_wlc.png', 
#           bbox_inches='tight')
#plt.savefig('/Users/mango/Desktop/wlc.pdf', bbox_inches='tight')

In [ ]:
wlc = pickle.load(open("/Users/mango/Desktop/ut170410/pca_WLC/final_LCs.pkl","rb"))

In [ ]:
wlc.keys()

## Corner Plots

### load data (takes a while)

In [ ]:
thefile = open(
    '/Users/mango/Desktop/ut170410/pca_wlc/MCMC_wl_5800.0_8800.0.pkl', 'rb')
ut170410_pca_MCMC_post = pickle.load(thefile)
thefile.close()

In [ ]:
def corner_plot(MCMC_data): # corner plots of wlc fits
    # join walker chains for individual params
    posteriors = MCMC_data
    t0_comb = np.concatenate(posteriors['t0'].values())
    sigma_r_comb = np.concatenate(posteriors['sigma_r'].values())
    sigma_w_comb = np.concatenate(posteriors['sigma_w'].values())
    RpRs_comb = np.concatenate(posteriors['RpRs'].values())
    F_comb = np.concatenate(posteriors['F'].values())
    w1_comb = np.concatenate(posteriors['w1'].values())

    # combine into 2D array
    posts = np.array([t0_comb, sigma_r_comb, sigma_w_comb, 
                      RpRs_comb, F_comb, w1_comb])
    samples = posts.transpose() # write rows as columns for corner plot
    # i.e. [[param1_0, param2_0, param3_0,...], 
    # [param1_1, param2_1, param3_1,...], ...]

    fig = plt.figure(figsize=(8, 10))

    corner.corner(samples, labels=['$t_0$', '$\sigma_r$', '$\sigma_w$',
                                   '$R_p/R_*$', '$F$', '$w_1$'],
                  show_titles=True)
    plt.show()
    #figure.title('ut150224 -- polynomial wavelet WLC')

In [ ]:
corner_plot(ut170410_pca_MCMC_post)
plt.savefig('/Users/mango/Desktop/pca_corner.pdf', bbox_inches='tight')
#plt.savefig('journal/Figures/20170725/ut170410_pca_wlc_corner.pdf', bbox_inches='tight')

## Binned plots 

In [ ]:
sns.palplot(sns.color_palette("cubehelix", 27))

In [ ]:
sns.palplot(sns.color_palette("Spectral", 27, desat=0.9))

In [ ]:
sns.palplot(sns.color_palette("Spectral", 50))

In [ ]:
# e.g. dataset: 'ut150309', model: 'poly', ofs: 0.01, const: 7 minutes
def plot_binned(ax, dataset, model, ofs): 
    # load data
    thefile = open(
    '/Users/mango/Dropbox/mercedes-group/wavelength_calibration/'+dataset+'/'+model+'_binned/final_LCs.pkl', 'rb')
    binned = pickle.load(thefile)
    thefile.close()
    
    t_rel = (binned['t'] - binned['t0'])*24
    
    ax.set_title('filler', color='w') # hacky way to make room for suptitle 
    ax.set_xlabel('filler', color='w') # hacky way to make room for fig x label

    N = len(binned['wbins']) # number of wavelength bins
    wmin = binned['wbins'].values()[0][0]
    wmax = binned['wbins'].values()[-1][-1]
    #colors = plt.cm.jet(np.linspace(0,1,wav_bins))
    colors = sns.color_palette("RdYlGn_r", N)

    offset = 0
    RpRs_list = []
    wav_list = []
    for i in range(N):
        ax.plot(t_rel, binned['detLC'][i] + offset, '.', color=colors[i])
        ax.plot(t_rel, binned['model'][i] + offset, '--', color='grey')
        wav_bin = [round(binned['wbins'][i][j], 3) for j in range(2)]
        ax.annotate(wav_bin, xy=(t_rel[0], 
                1.004*(binned['model'][i][0] + offset)),
                  fontsize=10)
        if binned['wbins'][i][0] in [5891.0, 6563.5, 7662.0, 7696.0, 8179.0, 8192.0]:
            ax.annotate(wav_bin, xy=(t_rel[0], 
                1.004*(binned['model'][i][0] + offset)),
                  fontsize=10, color='c')
            ax.plot(t_rel, binned['model'][i] + offset, 'c--')

        ax.set_ylabel('normalized flux + const')
        
        RpRs = np.min(binned['model'][i])
        RpRs_list.append(RpRs)
        wav_list.append(np.median(binned['wbins'][i]))
        
        offset += ofs
        
    #ax.set_yticklabels([])
    ax.set_ylim(0.9555, 1.207)
        
def plot_binned_resid(ax, dataset, model, ofs):
    # load data
    thefile = open(
    '/Users/mango/Dropbox/mercedes-group/wavelength_calibration/'+dataset+'/'+model+'_binned/final_LCs.pkl', 'rb')
    binned = pickle.load(thefile)
    thefile.close()
    
    t_rel = (binned['t'] - binned['t0'])*24
    N = len(binned['wbins'])
    colors = sns.color_palette("RdYlGn_r", N)
    
    offset = 0
    for i in range(N):
        ax.plot(t_rel, binned['residuals'][i] + offset, '.', color=colors[i])
        ax.plot(t_rel, np.zeros_like(t_rel) + offset, '--', color='grey')
        offset += ofs
        
    ax.set_yticklabels([])
    ax.set_ylim([0.9555-1, 1.207-1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8,10))

plot_binned(ax[0], 'ut170410', 'pca', 0.015)
plot_binned_resid(ax[1], 'ut170410', 'pca', 0.015)
#plt.savefig('/Users/mango/Desktop/pca_binned.pdf', bbox_inches='tight')
#plt.savefig('journal/Figures/20170709/ut170410_poly_binned.pdf', bbox_inches='tight')

#fig.suptitle('WASP43 -- ut150309 -- PCA wavelet binned -- ' + 
#    'Spectroscopic2 -- Gri-300-17.5',
#fontsize=12)
fig.suptitle('WASP43 -- ut170410 -- PCA wavelet binned -- ' + 
    'WB5600-9200 -- Gri-300-17.5',
fontsize=12)
fig.text(0.45, 0, '$t-t_0\ (hours)$')
fig.tight_layout()

#plt.savefig('/Users/mango/Desktop/ut150309_pca_binned.png',
#           bbox_inches='tight')
#plt.savefig('/Users/mango/Desktop/ut170410_binned.png', bbox_inches='tight')

In [ ]:
with open("WASP43/ut170410/pca_binned/final_LCs.pkl", "rb") as f:
    binned = pickle.load(f)

plt.figure()
for i in range(len(binned['detLC'])):
    #plt.cla()
    plt.plot(binned['detLC'][i], '.')
    plt.plot(binned['model'][i], '--', color='grey')

    #wav_bin = [round(binned['wbins'][i][j], 3) for j in range(2)]
    #plt.annotate("%i: %s" % (i,wav_bin), xy=(0, 1.004*(binned['model'][i][0])), fontsize=10)
plt.show()

# Transmission Spectra

### $R_p/R_s$ vs. $\lambda$

The posteriors are ordered by average wavelength $(\lambda_{min}+\lambda_{max})/2$ when saved to the pickle.

In [4]:
with open("WASP43/ut170410/pca_binned/posteriors.pkl", "rb") as f:
    post = pickle.load(f)

In [6]:
wav_avg = (post['wl_maxs'] + post['wl_mins']) / 2
binsize = post['wl_maxs'] - post['wl_mins']
RpRs_lci = post['RpRs']['LCIs'] # lower confidence interval
RpRs_med = post['RpRs']['meds'] # median
RpRs_uci = post['RpRs']['UCIs'] # upper confidence interval

RpRs_list = [RpRs_uci, RpRs_med, RpRs_lci]
labels    = ['UCI', 'med', 'LCI']
lb_clrs   = sns.color_palette('Accent', 3)

In [7]:
# since some of the bins are centered on a certain feature wavelength and just change in size,
# the average wavelength will be the same. This means that a list of the average 
# wavelengths will have copies whenever it hits one of the feature wavelenths. 
unique, counts = np.unique(wav_avg, return_counts=True) 
wav_counts = dict(zip(unique, counts)) # equivalent of a stem-and-leaf plot

# just get the feature wavelengths, which will have a count that is greater than 1.
wav_feat = {wav:count for wav,count in wav_counts.iteritems() if count != 1}

In [ ]:
from matplotlib import colors

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12,6), sharex=True, tight_layout=True)
#[im = ax.scatter(wav_avg, rprs, label=lb, c=binsize, cmap='viridis')
cmap = colors.ListedColormap(sns.color_palette('Set3', 10))
bounds=range(10, 110, 10)
norm = colors.BoundaryNorm(bounds, cmap.N)
for ax, rprs, lb, lb_cl in zip(axes.flatten(), RpRs_list, labels, lb_clrs):
    ax.plot(wav_avg, rprs, alpha=0.5, zorder=2, label=lb, c=lb_cl)
    im = ax.scatter(wav_avg, rprs, c=binsize, cmap=cmap, zorder=3, vmin=10,vmax=100)
    [ax.axvline(x=i, ls='--', c='grey', lw=0.5, zorder=1) for i in wav_feat]
    ymin = 0.99*np.min(rprs)
    ymax = 1.05*np.max(rprs)
    ax.set_ylim(ymin, ymax)
    ax.set_xlim(5855, 8244)
    ax.legend()
    #ax.set_title(lb)

# *left*, *bottom*, *width*,*height*
cbar_ax = fig.add_axes([0.059, 0.955, 0.929, 0.02])
cb = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.ax.xaxis.set_tick_params(pad=1)

#ax.set_xlabel(r'average wavelength $(\AA)$')
#ax.set_ylabel(r'$R_p / R_s$')
#ax.legend()

axes[2].set_xlabel(r'average wavelength $(\AA)$')
axes[1].set_ylabel(r' $R_p/R_s$')
fig.tight_layout()
plt.savefig("journal/Figures/20170823/ut170410_rprs_wav.pdf", bbox_incehs='tight')

### $R_p / R_s$ vs. binsize 

In [15]:
fig, axes = plt.subplots(2, 3, figsize=(8,5), sharex=True, sharey=True)
wav_diff = post['wl_maxs'] - post['wl_mins']
title_list = [r'$NaI-D$', r'$H\alpha$', r'$KI-a$', \
              r'$KI-b$', r'$NaI-8200a$', r'$NaI-8200b$']

# isolate user bins, assuming they increment in size by 10 up to 100
for ax, spec_idx, ttl in zip(axes.flatten(), np.arange(0, 60, 10), title_list): 
    ax.set_title(ttl)
    for rprs, lb_cl, lb in zip(RpRs_list, lb_clrs, labels):
        RpRs = rprs[((wav_diff <= 100) & (wav_diff % 10 == 0))][spec_idx:spec_idx+10]
        binsize_spec = wav_diff[((wav_diff <= 100) & (wav_diff % 10 == 0))][spec_idx:spec_idx+10]
        ax.plot(binsize_spec, RpRs, c=lb_cl, label=lb)
        
#fig.legend(lines, labels, loc = (0.5, 0), ncol=5 )

fig.text(0.01, 0.52, r'$R_p/R_s$', va='center', rotation='vertical')
fig.text(0.52, 0.01, r'$binsize\ (\AA)$', ha='center')

plt.legend(loc=9, bbox_to_anchor=(-2, -0.1), ncol=3)

fig.tight_layout(pad=2)

#plt.savefig('journal/Figures/20170823/ut170410_rprs_bin.pdf', bbox_inches='tight')